# <h1 align="center" id="heading">Explorando datos mundiales de COVID-19 </h1>

In [4]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import ipywidgets as widgets
import datetime

In [5]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [6]:
def formatData(path):
    #A. Utilizamos el nombre de país como índice principal
    df = pd.read_csv(path, index_col ='Country/Region')
    df.index.name = 'Pais'
    #B. Eliminamos las columnas Lat y Long
    df.drop(['Lat', 'Long'], axis = 1 ,inplace = True)
    #C. Usamos TimeStamps para los "nombres de columna" asociados a fechas
    lista_col = [df.columns[0]]
    #D. Reducimos la columna Province/State
    dict_dias_sum = {}                                            
    for i in range(1,df.shape[1],1):
        lista_col.append(pd.Timestamp(df.columns[i]).date()) #C. 
        dict_dias_sum[lista_col[i]] = np.sum # D.
    df.columns = lista_col #C.
    df = df.groupby('Pais').aggregate(dict_dias_sum) #D.
    return df

In [3]:
confirmados = formatData('data/covid19_confirmados.csv')

In [4]:
population = pd.read_csv('data/population_by_country_2020.csv', index_col = 'Country (or dependency)')
population.index.name = 'Pais'
population.drop(['Yearly Change', 'Net Change','Density (P/Km²)','Land Area (Km²)',
                'Migrants (net)', 'Fert. Rate', 'Med. Age','Urban Pop %', 'World Share'], 
                 axis = 1 ,inplace = True)

In [5]:
def mg(df, population):
    return pd.merge(df, population, how = 'inner', on= 'Pais')

In [6]:
confirmados = mg(confirmados, population)
recuperados = mg(formatData('data/covid19_recuperados.csv'),population)
decesos = mg(formatData('data/covid19_decesos.csv'),population)

In [9]:
#(a)Confirmados
confirmados_totales = confirmados.iloc[:, 0:460].sum(axis =1)
#(b)Cerrados:recuperados más decesos
cerrados = recuperados.iloc[:, 0:460].sum(axis =1)+decesos.iloc[:, 0:460].sum(axis =1)
#(c)Activos: confirmados menos cerrados
activos_totales = confirmados_totales - cerrados
decesos_totales = decesos.iloc[:, 0:460].sum(axis =1)
recuperados_totales = recuperados.iloc[:, 0:460].sum(axis =1)
poblacion_total = confirmados['Population (2020)'].sum()
tasa_incidencia = confirmados_totales*100000/poblacion_total
tasa_recuperacion = recuperados_totales*100000/poblacion_total
tasa_mortalidad = decesos_totales*100000/poblacion_total

OCDE = ["Austria", "Australia", "Belgium", "Canada", "Chile", "Colombia", "Denmark", "Estonia", 
        "Finland", "France", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Israel", "Italy", 
        "Japan", "Latvia", "Lithuania", "Luxembourg", "Mexico", "Netherlands", "New Zealand", "Norway", 
        "Poland", "Portugal", "Slovenia", "Spain", "Sweden", 
        "Switzerland", "Turkey", "US","United Kingdom", "Slovakia", "Czechia"]
# Fuente: https://www.oecd.org/newsroom/global-oecd-welcomes-colombia-as-its-37th-member.htm

### <h3 align="center" id="heading">Análisis temporal y comparativo  </h3> 

In [10]:
pais1_dropdown = widgets.Dropdown(options=list(OCDE),
                                description='País:',
                                disabled=False)

pais2_dropdown = widgets.Dropdown(options=list(OCDE),
                                description='País:',
                                disabled=False)

def generate_plot_confirmados(pais1, pais2):
    pais_x_1 = confirmados.loc[pais1].index
    pais_y_1 = confirmados.loc[pais1].values
    pais_x_1 = pais_x_1[0: len(pais_x_1)-1]
    pais_y_1 = pais_y_1[0: len(pais_y_1)-1]
    pais_x_2 = confirmados.loc[pais2].index
    pais_y_2 = confirmados.loc[pais2].values
    pais_x_2 = pais_x_2[0: len(pais_x_2)-1]
    pais_y_2 = pais_y_2[0: len(pais_y_2)-1]
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(pais_x_1,pais_y_1 ,label = pais1);ax.legend()
    ax.plot(pais_x_2,pais_y_2, label = pais2);ax.legend()
    ax.set(xlabel="Fecha", ylabel="Personas",title="Confirmados")
    
    plt.show()

    
def generate_plot_recuperados(pais1, pais2):
    pais_x_1 = recuperados.loc[pais1].index
    pais_y_1 = recuperados.loc[pais1].values
    pais_x_1 = pais_x_1[0: len(pais_x_1)-1]
    pais_y_1 = pais_y_1[0: len(pais_y_1)-1]
    pais_x_2 = recuperados.loc[pais2].index
    pais_y_2 = recuperados.loc[pais2].values
    pais_x_2 = pais_x_2[0: len(pais_x_2)-1]
    pais_y_2 = pais_y_2[0: len(pais_y_2)-1]
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(pais_x_1,pais_y_1,label = pais1);ax.legend()
    ax.plot(pais_x_2,pais_y_2,label = pais2);ax.legend()
    ax.set(xlabel="Fecha", ylabel="Personas",title="Recuperados")
    
    plt.show()
    
def generate_plot_decesos(pais1, pais2):
    pais_x_1 = decesos.loc[pais1].index
    pais_y_1 = decesos.loc[pais1].values
    pais_x_1 = pais_x_1[0: len(pais_x_1)-1]
    pais_y_1 = pais_y_1[0: len(pais_y_1)-1]
    pais_x_2 = decesos.loc[pais2].index
    pais_y_2 = decesos.loc[pais2].values
    pais_x_2 = pais_x_2[0: len(pais_x_2)-1]
    pais_y_2 = pais_y_2[0: len(pais_y_2)-1]
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(pais_x_1,pais_y_1,label = pais1);ax.legend()
    ax.plot(pais_x_2,pais_y_2,label = pais2);ax.legend()
    ax.set(xlabel="Fecha", ylabel="Personas",title="Decesos")
    plt.show()


def action_1():
    widgets.interact(generate_plot_confirmados, pais1=pais1_dropdown, pais2=pais2_dropdown);

def action_2():
    widgets.interact(generate_plot_recuperados, pais1=pais1_dropdown, pais2=pais2_dropdown);

def action_3():
    widgets.interact(generate_plot_decesos, pais1=pais1_dropdown, pais2=pais2_dropdown);

out1 = widgets.interactive(action_1)
out2 = widgets.interactive(action_2)
out3 = widgets.interactive(action_3)

tab  = widgets.Tab(children = [out1, out2, out3])
tab.set_title(0, 'Confirmados')
tab.set_title(1, 'Recuperados')
tab.set_title(2, 'Decesos')

display(tab)